<a href="https://colab.research.google.com/github/cche0214/HuggingFaceLLM/blob/main/02pipeline%E7%9A%84%E5%86%85%E9%83%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from transformers import pipeline

# pipeline集成了三个步骤，预处理，模型计算和后处理
classifier = pipeline("sentiment-analysis")
classifier(
    [
        "I've been waiting for a HuggingFace course my whole life.",
        "I hate this so much!",
    ]
)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


[{'label': 'POSITIVE', 'score': 0.9598050713539124},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

In [2]:
from transformers import AutoTokenizer

# 检查点是针对于架构的权重，这里的架构就是DistilBERT
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
# 这里的意思就是，找到这个检查点下的Tokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [5]:
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!"
]

# Tokenizer将原始的文本转换成tokenID（实际应该是原始文本->分词成token->token映射为tokenID
# return_tensors参数指定返回的tensor类型，可以是PyTorch、TensorFlow或者纯NumPy
# Transformers模型只接受张量输入
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")

In [7]:
# 输出包含两个键input_ids和attention_mask
# input_ids就是每个句子中token的ID
# 现在就是有了可以输入模型的数据，所以接下来下载模型
print(inputs)

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}


In [9]:
from transformers import AutoModel

# 这里并没有加载模型头，也就是不能得到情感分类任务的结果
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [10]:
outputs = model(**inputs)

# 输入句子，输出称为hidden states隐状态，这个维度很大有三个维度，隐状态也被称为模型头（输入下游任务）
# [BatchSize, SequenceLength, Hiddensize]
# 分别是一次处理的序列数量，表示序列（句子）的长度和每个模型输入的向量维度
print(outputs.last_hidden_state.shape)

torch.Size([2, 16, 768])


In [11]:
print(outputs)

BaseModelOutput(last_hidden_state=tensor([[[-0.1798,  0.2333,  0.6321,  ..., -0.3017,  0.5008,  0.1481],
         [ 0.2758,  0.6497,  0.3200,  ..., -0.0760,  0.5136,  0.1329],
         [ 0.9046,  0.0985,  0.2950,  ...,  0.3352, -0.1407, -0.6464],
         ...,
         [ 0.1466,  0.5661,  0.3235,  ..., -0.3376,  0.5100, -0.0561],
         [ 0.7500,  0.0487,  0.1738,  ...,  0.4684,  0.0030, -0.6084],
         [ 0.0519,  0.3729,  0.5223,  ...,  0.3584,  0.6500, -0.3883]],

        [[-0.2937,  0.7283, -0.1497,  ..., -0.1187, -1.0227, -0.0422],
         [-0.2206,  0.9384, -0.0951,  ..., -0.3643, -0.6605,  0.2407],
         [-0.1536,  0.8988, -0.0728,  ..., -0.2189, -0.8528,  0.0710],
         ...,
         [-0.3017,  0.9002, -0.0200,  ..., -0.1082, -0.8412, -0.0861],
         [-0.3338,  0.9674, -0.0729,  ..., -0.1952, -0.8181, -0.0634],
         [-0.3454,  0.8824, -0.0426,  ..., -0.0993, -0.8329, -0.1065]]],
       grad_fn=<NativeLayerNormBackward0>), hidden_states=None, attentions=None)


In [14]:
# 现在要导入一个带有序列分类头的模型，也就是情感分类模型
from transformers import AutoModelForSequenceClassification

# 这里的checkpoint跟单纯导入AutoModle一样，原因是两者的区别只是有没有下游的模型头
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)


In [15]:
# 这里的维度小了很多，因为模型头已经接受了前面的高维向量作为输入，输出包含两个值（每种标签一个的）向量
print(outputs.logits.shape)

torch.Size([2, 2])


In [16]:
print(outputs)

SequenceClassifierOutput(loss=None, logits=tensor([[-1.5607,  1.6123],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


In [17]:
# 预测出来的暂时还是对数几率，是模型最后一层输出的原始的、未标准化的分数
# 因此要知道最后的概率，需要经过softmax层，所有Transformers模型的输出都是logits
print(outputs.logits)

tensor([[-1.5607,  1.6123],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward0>)


In [18]:
import torch

# 经过softmax层之后才得到概率，现在需要查看模型看每个概率对应哪种标签
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

tensor([[4.0195e-02, 9.5981e-01],
        [9.9946e-01, 5.4418e-04]], grad_fn=<SoftmaxBackward0>)


In [19]:
# 现在知道第一个是负面的概率，第二个是积极的概率
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}